In [ ]:
import numpy as np
import pandas as pd
import re
import emot
import nltk
import wordsegment
import optuna
import pymorphy2
from nltk.tokenize import RegexpTokenizer
from tqdm import notebook
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from optuna.samplers import TPESampler

nltk.download('stopwords')
nltk.download('punkt')
wordsegment.load()

TOKEN_PATTERN = "[а-яёa-z]+"

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikolaystepanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikolaystepanov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [ ]:
train_df

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False
...,...,...,...,...
135304,135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,False
135305,135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....,False
135306,135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati...",False
135307,135307,wowcream.ru,Купить It's Skin Сыворотка питательная Power 1...,False


In [ ]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


# Preprocessing

In [ ]:
class TitlePreprocessor:

    def __init__(self, corpus, token_pattern):
        self.corpus = corpus
        self.tokenizer = RegexpTokenizer(token_pattern)
        self.texts = []
        self.lemmatizer_cache = {}
        self.lemmatizer = pymorphy2.MorphAnalyzer()
        self.emot_handler = emot.core.emot()

    def replace_emoji(self, text):
        emojies_res = self.emot_handler.emoji(text)
        emoticons_res = self.emot_handler.emoticons(text)
        if emojies_res['flag']:
            values = emojies_res['value']
            means = emojies_res['mean']
            for i in range(len(values)):
                text = text.replace(values[i], " " + means[i] + " ")
        if (emoticons_res['flag']):
            values = emoticons_res['value']
            means = emoticons_res['mean']
            for i in range(len(values)):
                text = text.replace(values[i], " " + means[i] + " ")
        return text

    def handle_emoji(self):
        self.corpus = [self.replace_emoji(text) for text in self.corpus]

    def tokenize(self):
        self.texts = [self.tokenizer.tokenize(text.lower()) for text in self.corpus]

    def lemmatize_simple(self, token):
        if self.lemmatizer.word_is_known(token):
            if token not in self.lemmatizer_cache:
                self.lemmatizer_cache[token] = self.lemmatizer.parse(token)[0].normal_form
            return self.lemmatizer_cache[token]
        return token

    def lemmatize(self):
        self.texts = [[self.lemmatize_simple(token) for token in text]
                      for text in notebook.tqdm(self.texts)]

    def clear(self):
        stopword_set = set(nltk.corpus.stopwords.words('russian') + nltk.corpus.stopwords.words('english'))
        self.texts = [[token for token in text if token not in stopword_set]
                          for text in self.texts]

    def process(self):
        self.handle_emoji()
        self.tokenize()
        self.lemmatize()
        self.clear()
        return [" ".join(text) for text in self.texts]

In [ ]:
class UrlPreprocessor:
    def __init__(self, corpus):
        self.corpus = corpus
        self.texts = []

    def segment(self):
        self.texts = [[seg for part in url for seg in wordsegment.segment(part)] for url in self.texts]

    def tokenize(self):
        self.texts = [re.split("-|\.", url) for url in self.corpus]

    def clear(self):
        stopword_set = {'ru', 'com', 'www', 'md', 'org', 'online'}
        self.texts = [[token for token in url if token not in stopword_set]
                          for url in self.texts]

    def process(self):
        self.tokenize()
        self.segment()
        self.clear()
        return [" ".join(text) for text in self.texts]

In [ ]:
class TotalPreprocessor:
    def __init__(self, X):
        self.X = X
        self.title_preprocessor = TitlePreprocessor(self.X["title"].values, TOKEN_PATTERN)
        self.url_preprocessor = UrlPreprocessor(self.X["url"].values)

    def process(self):
        self.X["title"] = self.title_preprocessor.process()
        self.X["url"] = self.url_preprocessor.process()
        return self.X

In [ ]:
y_train = train_df["target"].astype(int).values
X_train = train_df.drop(["id", "target"], axis=1)
X_test = test_df.drop(["id"], axis=1)
X_train

,url,title
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц..."
1,www.kp.by,Эта песня стала известна многим телезрителям б...
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...
3,colorbox.spb.ru,Не Беси Меня Картинки
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...
...,...,...
135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...
135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....
135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati..."
135307,wowcream.ru,Купить It's Skin Сыворотка питательная Power 1...


In [ ]:
X_train_preprocessed = TotalPreprocessor(X_train).process()
X_train_preprocessed

  0%|          | 0/135309 [00:00<?, ?it/s]

,url,title
0,m kp,экс министр экономика молдова глава мидэи цель...
1,kp by,песня стать известный многий телезритель благо...
2,fan serials tv,банши сезон серия бремя красота смотреть онлайн
3,colorbox spb,бесить картинка
4,tula sport,новомосковск сыграть следж хоккеист алексински...
...,...,...
135304,mail,пора тюльпан турецкий сериал русский язык резу...
135305,ntv,остросюжетный сериал шеф игра повышение серия
135306,top classic cars for sale,plymouth special deluxe hot rod automatic smal...
135307,wow cream,купить skin сыворотка питательный power formul...


# Feature Engineering

In [ ]:
checked_titles = X_train_preprocessed[y_train == 1].title.values
tag_words_title_counter = Counter([token for doc in checked_titles for token in doc.split()])
tag_words_title = [i[0] for i in tag_words_title_counter.most_common(100)]
tag_words_title[:10]

['порно',
 'porn',
 'видео',
 'sex',
 'videos',
 'онлайн',
 'hd',
 'video',
 'секс',
 'com']

In [ ]:
checked_url = X_train_preprocessed[y_train == 1].url.values
tag_words_url_counter = Counter([token for doc in checked_url for token in doc.split()])
tag_words_url = [i[0] for i in tag_words_url_counter.most_common(50)]
tag_words_url

['porno',
 'net',
 'porn',
 'sex',
 'x',
 'tv',
 'le',
 'daft',
 'biq',
 'm',
 'video',
 'cc',
 'tube',
 'xxx',
 'me',
 'girls',
 'mobi',
 'info',
 'perfect',
 'videos',
 'club',
 'an',
 'en',
 'censored',
 'pro',
 'y',
 'a',
 'er',
 'uk',
 'hd',
 'k',
 'xyz',
 'e',
 '24',
 'top',
 'i',
 'comics',
 'ero',
 'co',
 'tubs',
 'exer',
 'on',
 'bobs',
 'free',
 'do',
 'wap',
 'rus',
 'life',
 'f',
 'ch']

In [ ]:
def TagData(X):
    taged_X = X.copy()
    tags = np.zeros(taged_X.shape[0])
    titles = taged_X['title'].values
    urls = taged_X['url'].values
    for i in range(taged_X.shape[0]):
        tags[i] = len([i for i in titles[i].split() if i in tag_words_title] +
                      [i for i in urls[i].split() if i in tag_words_url])
    taged_X['tags_count'] = tags
    return taged_X

In [ ]:
taged_X_train = TagData(X_train_preprocessed)
taged_X_train

# CatBoost(simple)

In [ ]:
new_X_train, X_val, new_y_train, y_val = train_test_split(taged_X_train, y_train, test_size=0.25, random_state=42)
new_X_train

,url,title,tags_count
109524,new inform,американский сми оценить азиатский разворот по...,0.0
13548,avatar ko,аватар девушка чужой аватар тигр фото прикол а...,2.0
62727,family guy fox fan tv,серия сезон мультсериал гриффины английский су...,5.0
114853,yoox,модель футляр женщина anna rachele yoox россия,0.0
60402,drive 2,бить руль торможение скорость км chevrolet ave...,0.0
...,...,...,...
110268,porno photo pro,смотреть бритый вагин фото,4.0
119879,multi listing su,продать трехкомнатную вторичку мкр олимпийский...,0.0
103694,digital 1k by,sharp dg купить минск k,0.0
131932,citi link,купить web камера logitech conferencecam rally...,0.0


In [ ]:
train_pool = Pool(
    data=new_X_train,
    label=new_y_train,
    text_features=["title", "url"]
)

val_pool = Pool(
    data=X_val,
    label=y_val,
    text_features=["title", "url"]
)

/Users/nikolaystepanov/opt/anaconda3/envs/mvp/lib/python3.10/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [ ]:
params = {
    "iterations": 100,
    "depth": 8,
    "loss_function": "Logloss",
    "eval_metric": "F1",
    "verbose": False,
    'random_seed': 42,
}

model = CatBoostClassifier(**params)
model.fit(train_pool)

In [ ]:
y_pred = model.predict(val_pool)
res = f1_score(y_val, y_pred)
res

0.9740055616007738

In [ ]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,title,83.342597
1,url,9.127899
2,tags_count,7.529504


# Hyperparameters tuning

In [ ]:
from hyperopt import hp, fmin, tpe

def hyperopt_objective(params):
    print(params)
    model = CatBoostClassifier(**params)
    model.fit(train_pool)
    y_pred = model.predict(val_pool)
    return -f1_score(y_val, y_pred)

space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'depth': hp.randint('depth', 3, 10),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
    'boosting_type': hp.choice('boosting_type', ['Ordered', 'Plain']),
    "iterations": 100,
    "loss_function": "Logloss",
    "eval_metric": "F1",
    'verbose': False,
}

best = fmin(hyperopt_objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    rstate=np.random.default_rng(42))

{'boosting_type': 'Plain', 'depth': 6, 'eval_metric': 'F1', 'iterations': 100, 'l2_leaf_reg': 6.58107097908977, 'learning_rate': 0.0832289866876259, 'loss_function': 'Logloss', 'random_seed': 42, 'verbose': False}
{'boosting_type': 'Plain', 'depth': 9, 'eval_metric': 'F1', 'iterations': 100, 'l2_leaf_reg': 6.474242459540229, 'learning_rate': 0.03703792193114575, 'loss_function': 'Logloss', 'random_seed': 42, 'verbose': False}
{'boosting_type': 'Plain', 'depth': 4, 'eval_metric': 'F1', 'iterations': 100, 'l2_leaf_reg': 1.662949229054717, 'learning_rate': 0.0232266508556525, 'loss_function': 'Logloss', 'random_seed': 42, 'verbose': False}
{'boosting_type': 'Plain', 'depth': 7, 'eval_metric': 'F1', 'iterations': 100, 'l2_leaf_reg': 2.909954052940017, 'learning_rate': 0.046339990323578134, 'loss_function': 'Logloss', 'random_seed': 42, 'verbose': False}
{'boosting_type': 'Ordered', 'depth': 3, 'eval_metric': 'F1', 'iterations': 100, 'l2_leaf_reg': 1.8698018110930823, 'learning_rate': 0.055

# Submit

In [ ]:
X_test_preprocessed = TotalPreprocessor(X_test).process()
taged_X_test = TagData(X_test_preprocessed)

  0%|          | 0/165378 [00:00<?, ?it/s]

In [ ]:
test_pool = Pool(
    data=taged_X_test,
    text_features=["title", "url"]
)

test_df["target"] = model.predict(test_pool).astype(bool)

test_df[["id", "target"]].to_csv("catboost_baseline.csv", index=False)

!cat catboost_baseline.csv | head

/Users/nikolaystepanov/opt/anaconda3/envs/mvp/lib/python3.10/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: stdout: Broken pipe
